**The Game Sequence**

The player is numbered from 1, 2, ... the global variables *cards_on_board* and *the_cards* are what you get in the box when you buy Sequence. That is the board and cards. Also you will get discs in your player's colour.

This notebook contains the following functions:

  1. **initGame**(): returns the initial *discs_on_board* and deals out cards to the players stored in the list of lists called *hand*.
  2. **isTerminal**(discs_on_board, player): determines if the lookahead of *player* is terminal, if so this *player* wins
  3. **getMoves**(discs_on_board, hand, player) will return a list of tuples ($i,j$) for cards in hand, for normal cards (*legal_moves*) one eyed jacks (*legal_moves_1J*) and two eyed jacks (*legal_moves_2J*)
  4. **drawCard**(deck, hand, card_played) will return a new *hand[player-1]* for *player* and updated the deck by removing the top card. The *card_played* by the *player* is removed from its hand.
  5. **pretty_print**()

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from SequenceGame import SequenceEnv

In [ ]:
env = SequenceEnv()

print(env.discs_on_board)

env.play_full_game()

In [ ]:
board_rows = 10
board_cols = 10

heuristic_1_table = np.zeros((n, 10, 10))

def heuristic_1(temp_board, hand, player, i, j):
  # Namminamm
  # temp_board: discs_on_board, nema player í stað -1 í hornunum

  s = 0

  # Horizontal
  mint = max(-4, -j)
  maxt = min(4, board_cols-1-j)
  t = mint
  while t < 0:
    if temp_board[i][j+t] != player and temp_board[i][j+t] != 0:
      mint = t+1
  t = maxt
  while t > 0:
    if temp_board[i][j+t] != player and temp_board[i][j+t] != 0:
      maxt = t-1

  range = maxt - mint + 1
  if range >= 5:
    t = mint
    while t <= maxt:
      if temp_board[i][j+t] == player:
        s += min(min(t - mint, maxt - t), range - 5) + 1
      t -= 1

  # Vertical
  mint = max(-4, -i)
  maxt = min(4, board_rows-1-i)
  t = mint
  while t < 0:
    if temp_board[i+t][j] != player and temp_board[i+t][j] != 0:
      mint = t+1
  t = maxt
  while t > 0:
    if temp_board[i+t][j] != player and temp_board[i+t][j] != 0:
      maxt = t-1

  range = maxt - mint + 1
  if range >= 5:
    t = mint
    while t <= maxt:
      if temp_board[i+t][j] == player:
        s += min(min(t - mint, maxt - t), range - 5) + 1
        t -= 1
  
  # Main diagonal
  mint = max(-4, max(-i, -j))
  maxt = min(4, min(board_rows-1-i, board_rows-1-j))
  t = mint
  while t < 0:
    if temp_board[i+t][j+t] != player and temp_board[i+t][j+t] != 0:
      mint = t+1
  t = maxt
  while t > 0:
    if temp_board[i+t][j+t] != player and temp_board[i+t][j+t] != 0:
      maxt = t-1

  range = maxt - mint + 1
  if range >= 5:
    t = mint
    while t <= maxt:
      if temp_board[i+t][j+t] == player:
        s += min(min(t - mint, maxt - t), range - 5) + 1
        t -= 1
  
  # Antidiagonal
  mint = max(-4, max(-i, board_rows-1-j))
  maxt = min(4, min(board_rows-1-i, -j))
  t = mint
  while t < 0:
    if temp_board[i+t][j-t] != player and temp_board[i+t][j-t] != 0:
      mint = t+1
  t = maxt
  while t > 0:
    if temp_board[i+t][j-t] != player and temp_board[i+t][j-t] != 0:
      maxt = t-1

  range = maxt - mint + 1
  if range >= 5:
    t = mint
    while t <= maxt:
      if temp_board[i+t][j-t] == player:
        s += min(min(t - mint, maxt - t), range - 5) + 1
        t -= 1

def update_heuristic_1(discs_on_board, hand, player, i, j):
  # Uppfæra Heuristic 1 eftir leik

  temp_board = discs_on_board.copy()
  temp_board[temp_board == -1] = player

  for p in range(n):
    if p+1 == player:
      heuristic_1_table[p][i][j] = heuristic_1(discs_on_board, hand, player, i, j)
    else:
      heuristic_1_table[p][i][j] = 0

  # Horizontal
  for p in range(n):
    for t in range(1, 5):
      if temp_board[i+t][j] != 0 and temp_board[i+t][j] != p:
        break
      heuristic_1_table[player-1][i+t][j] = heuristic_1(temp_board, hand, player, i+t, j)
    for t in range(1, 5):
      if temp_board[i-t][j] != 0 and temp_board[i-t][j] != p:
        break
      heuristic_1_table[player-1][i-t][j] = heuristic_1(temp_board, hand, player, i-t, j)

In [ ]:
# ELÍAS

def one_hot_encoding(board, hand):
  temp_board = board.copy().flatten()
  # ATH. Segi að það sé enginn leikmaður á hornunum. Þetta þarf að endurskoða, kannski sleppa.
  temp_board = temp_board[temp_board != -1]
  # One hot encoding á borði
  one_hot = torch.nn.functional.one_hot(torch.tensor(temp_board).long(), num_classes = n + 1)
  # Flet borðið út. Hér væri líka hægt að halda forminu og breyta kóðun á hönd
  one_hot = one_hot.flatten()
  # Kóða höndina eins og talað var um
  hond = np.zeros((50), dtype = np.int32)
  np.add.at(hond, hand, 1)
  # Skeyti hönd aftan við borð, fjöldi spila á indexi. Ef sett er hand í stað hönd skeytist nr. spila við, mögulega jafngóð kóðun
  return torch.cat((one_hot, torch.tensor(hond)))